In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
from scipy.stats import pearsonr
from multiprocessing import Pool
import time

# Function to generate an Ornstein-Uhlenbeck (OU) process
def generate_OU_process(tau, dt, T):
    """
    Generate an Ornstein-Uhlenbeck process with zero mean and unit variance.

    Parameters:
    - tau: Time constant of the OU process.
    - dt: Time step size.
    - T: Total time duration.

    Returns:
    - y: Array containing the OU process values.
    """
    N = int(T / dt)
    y = np.zeros(N)
    for i_t in range(1, N):
        y[i_t] = (1 - dt / tau) * y[i_t - 1] + np.sqrt(2 * dt / tau) * np.random.randn()
    return y

# Function to merge clusters based on activity correlations
def merge_clusters(clusters, data):
    """
    Merge pairs of clusters in the cluster list to form new clusters of double the size.

    Parameters:
    - clusters: List of clusters; clusters[i] contains neuron indices in cluster i.
    - data: Original data matrix used to compute correlations.

    Returns:
    - new_clusters: List of merged clusters.
    """
    num_clusters = len(clusters)
    new_clusters = []
    merged_flags = [False] * num_clusters  # Flags to mark merged clusters

    # Compute the activity (sum of neuron activities within the cluster) for each cluster
    cluster_activities = []
    for cluster_indices in clusters:
        cluster_data = data[:, cluster_indices]
        # Sum activities of neurons
        activity = np.sum(cluster_data, axis=1)
        # Normalize to have a mean of 1 for non-zero entries
        nonzero_indices = activity != 0
        if np.any(nonzero_indices):
            mean_nonzero = np.mean(activity[nonzero_indices])
            activity[nonzero_indices] = activity[nonzero_indices] / mean_nonzero
        cluster_activities.append(activity)

    # Compute correlation matrix between clusters
    num_activities = len(cluster_activities)
    corr_matrix = np.full((num_activities, num_activities), -np.inf)  # Initialize to -inf to avoid self-correlation

    for i in range(num_activities):
        for j in range(i + 1, num_activities):
            corr_value, _ = pearsonr(cluster_activities[i], cluster_activities[j])
            corr_matrix[i, j] = corr_value

    # Begin merging process
    while np.sum(~np.array(merged_flags)) >= 2:
        # Find the pair of clusters with the highest correlation among unmerged clusters
        available_indices = [idx for idx, flag in enumerate(merged_flags) if not flag]
        sub_corr_matrix = corr_matrix[np.ix_(available_indices, available_indices)]

        # Find the maximum correlation and its indices
        idx = np.argmax(sub_corr_matrix)
        row, col = np.unravel_index(idx, sub_corr_matrix.shape)
        i = available_indices[row]
        j = available_indices[col]

        # Merge clusters i and j
        merged_cluster = clusters[i] + clusters[j]
        new_clusters.append(merged_cluster)

        # Mark clusters as merged
        merged_flags[i] = True
        merged_flags[j] = True

    # If there are remaining unmerged clusters, discard them
    if any(not flag for flag in merged_flags):
        num_discarded = sum(not flag for flag in merged_flags)
        print(f'Discarded {num_discarded} unmerged clusters to meet cluster size requirement.')

    return new_clusters

# Function to compute average eigenvalues and total variance for clusters
def compute_average_eigenvalues(original_data, clusters, K):
    """
    Compute the average eigenvalue spectrum and total variance for given clusters.

    Parameters:
    - original_data: Original data matrix [num_time_steps x num_neurons_initial].
    - clusters: List of clusters; clusters[i] contains neuron indices in cluster i.
    - K: Current cluster size.

    Returns:
    - mean_eigenvalues: Mean eigenvalues [1 x K].
    - rank_K: Rank/K values [1 x K].
    - total_variances: Total variance for each cluster [num_clusters x 1].
    """
    num_clusters = len(clusters)

    # Store eigenvalues for each cluster
    eigenvalues_all_clusters = np.zeros((num_clusters, K))
    total_variances = np.zeros(num_clusters)

    for c in range(num_clusters):
        cluster_indices = clusters[c]  # Indices of neurons in the current cluster

        # Check if cluster size equals K
        if len(cluster_indices) != K:
            raise ValueError(f'Cluster size not equal to K = {K}, actual size is {len(cluster_indices)}.')

        # Get spike data for the cluster
        cluster_spike_data = original_data[:, cluster_indices]  # [num_time_steps x K]

        # Remove mean
        cluster_spike_data = cluster_spike_data - np.mean(cluster_spike_data, axis=0)

        # Compute covariance matrix
        cov_matrix = np.cov(cluster_spike_data, rowvar=False)  # [K x K]

        # Compute eigenvalues and sort in descending order
        eigenvalues = np.linalg.eigvalsh(cov_matrix)
        eigenvalues_sorted = np.sort(eigenvalues)[::-1]

        # Store eigenvalues
        eigenvalues_all_clusters[c, :] = eigenvalues_sorted

        # Compute total variance (sum of eigenvalues)
        total_variances[c] = np.sum(eigenvalues_sorted)

    # For each rank, compute the mean eigenvalue across all clusters
    mean_eigenvalues = np.mean(eigenvalues_all_clusters, axis=0)
    rank_K = np.arange(1, K + 1) / K
    return mean_eigenvalues, rank_K, total_variances

# Main program
if __name__ == '__main__':
    start_time = time.time()

    # Define ranges for eta and epsilon
    eta_vals = np.arange(5, 6 + 0.5, 0.5)  # eta from 5 to 6 with step size 0.5
    epsilon_vals = np.arange(-18, 18 + 1, 1)  # epsilon from -18 to 18 with step size 1
    num_eta = len(eta_vals)
    num_eps = len(epsilon_vals)

    # Set Nf
    Nf = 5

    # Initialize matrices to store R^2 values and slope z
    R2_values = np.full((num_eta, num_eps), np.nan)  # R^2 values for each (eta, epsilon)
    z_values = np.full((num_eta, num_eps), np.nan)   # Slope z for each (eta, epsilon)

    # Pre-generate h_m_finitetau and norm_J_vals
    bin_size = 2
    max_dim = Nf
    N_T = 100000  # Number of time steps
    num_neur = 512
    bin_size_xc = bin_size
    bin_size0 = 2
    finite_tau = 1000  # Time constant
    dt = 1  # Time step size

    # Generate h_m_finitetau
    h_m_finitetau = np.zeros((int(N_T * (bin_size_xc / bin_size0)), max_dim))
    for i_d in range(max_dim):
        h_m_finitetau[:, i_d] = generate_OU_process(finite_tau, dt, N_T * (bin_size_xc / bin_size0))

    # Generate random weights norm_J_vals
    norm_J_vals = np.random.randn(num_neur, max_dim).T  # Transpose to match dimensions

    # Create folder to save images (absolute path)
    output_folder = os.path.join(os.getcwd(), 'Eigenvalue_Plots_Nf_5')
    os.makedirs(output_folder, exist_ok=True)  # Safe to call in parallel environment

    # Define function to process each eta value
    def process_eta(i_eta):
        this_eta_val = eta_vals[i_eta]

        # Initialize arrays to store results for current eta
        R2_values_i_eta = np.full(num_eps, np.nan)
        z_values_i_eta = np.full(num_eps, np.nan)

        for i_eps in range(num_eps):
            this_eps_val = epsilon_vals[i_eps]
            print(f'Processing eta = {this_eta_val:.2f}, epsilon = {this_eps_val:.2f}')

            # Simulate data using the same J and h
            def p_s_i_fun(h_m, J_m, eta_val, epsilon_val):
                return 1.0 / (1 + np.exp(-eta_val * (h_m @ J_m) - epsilon_val))

            # Compute p_s_i_vals_ft using pre-generated h_m_finitetau and norm_J_vals
            p_s_i_vals_ft = p_s_i_fun(h_m_finitetau[:, :max_dim], norm_J_vals, this_eta_val, this_eps_val)

            # Ensure p_s_i_vals_ft is within [0, 1]
            p_s_i_vals_ft = np.clip(p_s_i_vals_ft, np.finfo(float).eps, 1 - np.finfo(float).eps)

            # Generate spike counts
            sc_i_vals_ft = np.random.rand(*p_s_i_vals_ft.shape) < p_s_i_vals_ft
            binned_sc_i_ft = np.sum(sc_i_vals_ft.reshape(int(bin_size_xc / bin_size0), N_T, num_neur), axis=0)

            # Remove neurons with low activity
            nonsilent_cells = np.sum(binned_sc_i_ft, axis=0) > 5
            binned_sc_i_ft = binned_sc_i_ft[:, nonsilent_cells]

            data = binned_sc_i_ft  # [num_time_steps x num_neurons_initial]
            num_time_steps, num_neurons_initial = data.shape
            print(f'The size of spike count matrix is {num_time_steps} x {num_neurons_initial}.')

            # Begin coarse-graining code
            # Set K values
            target_K_values = [2, 4, 8, 16, 32, 64, 128]
            num_K = len(target_K_values)

            # Initialize variables
            clusters_rounds = [None] * num_K       # Clusters for each round
            mean_eigenvalues_by_K = [None] * num_K  # Mean eigenvalues for each round
            rank_K = [None] * num_K                # Rank/K for each round
            slopes_z = np.full(num_K, np.nan)      # Slope z for each K
            mean_variances_by_K = np.full(num_K, np.nan)  # Mean variance for each round

            # Initialize clustering
            initial_clusters = [[i] for i in range(num_neurons_initial)]  # clusters[i] contains neuron indices
            current_clusters = initial_clusters.copy()

            # Begin coarse-graining rounds
            for round_idx in range(num_K):
                K = target_K_values[round_idx]
                print(f'\n===== Round {round_idx + 1}: K = {K} =====')

                # Calculate the number of clusters needed and the number of merges
                previous_cluster_size = K // 2  # Previous cluster size
                num_previous_clusters = len(current_clusters)

                # Check if there are enough clusters to merge
                if previous_cluster_size < 1 or num_previous_clusters % 2 != 0:
                    # If the number of clusters is not even, discard extra clusters
                    num_clusters_to_use = (num_previous_clusters // 2) * 2
                    print(f'Discarded {num_previous_clusters - num_clusters_to_use} clusters to meet cluster size requirement.')
                    current_clusters = current_clusters[:num_clusters_to_use]

                # Perform merging
                new_clusters = merge_clusters(current_clusters, data)

                # Store clusters for this round
                clusters_rounds[round_idx] = new_clusters

                # Compute average eigenvalues and total variance for this round
                mean_eigenvalues, rank_K_round, total_variances = compute_average_eigenvalues(data, new_clusters, K)
                mean_eigenvalues_by_K[round_idx] = mean_eigenvalues
                rank_K[round_idx] = rank_K_round

                # Compute mean variance
                mean_variance = np.mean(total_variances)
                mean_variances_by_K[round_idx] = mean_variance

                print(f'Round {round_idx + 1} completed: formed {len(new_clusters)} clusters, average eigenvalues and variance calculation completed.')

                # Prepare for next round
                current_clusters = new_clusters.copy()

            # Plot and save eigenvalue images, compute R^2 and z
            # Fit for K >= 32
            idx_K_list = [idx for idx, K_val in enumerate(target_K_values) if K_val >= 32]  # Indices where K >= 32
            combined_log_rank_K = []
            combined_log_eigenvalues = []

            # Create a figure for current eta and epsilon
            fig, ax = plt.subplots()

            for idx in idx_K_list:
                K_i = target_K_values[idx]
                if mean_eigenvalues_by_K[idx] is None:
                    continue  # Skip if no data
                rank_K_round = rank_K[idx]
                mean_eigenvalue = mean_eigenvalues_by_K[idx]

                # Remove NaN values
                valid_indices = ~np.isnan(mean_eigenvalue)
                rank_K_round = rank_K_round[valid_indices]
                mean_eigenvalue = mean_eigenvalue[valid_indices]

                # Ensure they are arrays
                rank_K_round = rank_K_round.flatten()
                mean_eigenvalue = mean_eigenvalue.flatten()

                # Logarithmic values
                log_rank_K = np.log(rank_K_round)
                log_eigenvalues = np.log(mean_eigenvalue)

                # Determine fitting range (Rank/K >= 0 & Rank/K <= 0.3)
                fit_range = (rank_K_round >= 0) & (rank_K_round <= 0.3)

                # Check if there are enough data points
                if np.sum(fit_range) < 2:
                    print(f'Warning: K = {K_i} does not have enough data points for fitting.')
                    slopes_z[idx] = np.nan
                    continue

                # Linear fit
                p = np.polyfit(log_rank_K[fit_range], log_eigenvalues[fit_range], 1)

                # Record slope z
                slopes_z[idx] = p[0]

                # Plot curve
                ax.plot(rank_K_round, mean_eigenvalue, 'o-', label=f'K = {K_i}')

                # Plot fitted line
                fitted_line = np.exp(np.polyval(p, log_rank_K[fit_range]))
                ax.plot(rank_K_round[fit_range], fitted_line, '--', linewidth=1)

                # Collect data for combined fit
                combined_log_rank_K.extend(log_rank_K[fit_range])
                combined_log_eigenvalues.extend(log_eigenvalues[fit_range])

            # Perform combined fit over all K values
            if len(combined_log_rank_K) >= 2:
                # Linear fit
                p_combined = np.polyfit(combined_log_rank_K, combined_log_eigenvalues, 1)
                z_combined = p_combined[0]
                log_eigenvalues_fit_combined = np.polyval(p_combined, combined_log_rank_K)

                # Compute combined R^2
                residuals_combined = np.array(combined_log_eigenvalues) - log_eigenvalues_fit_combined
                SS_res_combined = np.sum(residuals_combined ** 2)
                SS_tot_combined = np.sum((np.array(combined_log_eigenvalues) - np.mean(combined_log_eigenvalues)) ** 2)
                R2_combined = 1 - SS_res_combined / SS_tot_combined

                # Save R2 and z
                R2_values_i_eta[i_eps] = R2_combined
                z_values_i_eta[i_eps] = z_combined
            else:
                R2_values_i_eta[i_eps] = np.nan
                z_values_i_eta[i_eps] = np.nan

            # Finalize plot
            ax.set_xlabel('Rank / K')
            ax.set_ylabel('Mean Eigenvalue')
            ax.set_title(f'Mean Eigenvalue vs Rank / K (eta={this_eta_val:.2f}, epsilon={this_eps_val:.2f})')
            ax.set_xscale('log')
            ax.set_yscale('log')
            ax.legend(loc='best')
            ax.grid(True)

            # Save image
            filename = f'Eigenvalue_vs_RankK_eta{this_eta_val:.2f}_eps{this_eps_val:.2f}.png'
            filename = filename.replace('.', '_').replace('-', 'neg')
            plt.savefig(os.path.join(output_folder, filename))
            plt.close(fig)

            print(f'Eta = {this_eta_val:.2f}, Epsilon = {this_eps_val:.2f}, Combined R^2 = {R2_values_i_eta[i_eps]:.4f}, z = {z_values_i_eta[i_eps]:.4f}')

        # Return results for current eta
        return R2_values_i_eta, z_values_i_eta

    # Begin parallel processing over eta values
    if __name__ == '__main__':
        # Use multiprocessing pool for parallel computation
        num_processes = min(6, os.cpu_count())  # Adjust number of processes based on system
        with Pool(processes=num_processes) as pool:
            # Map the process_eta function to each i_eta
            results = pool.map(process_eta, range(num_eta))

        # Collect results
        for i_eta, (R2_values_i_eta, z_values_i_eta) in enumerate(results):
            R2_values[i_eta, :] = R2_values_i_eta
            z_values[i_eta, :] = z_values_i_eta

        # Save the R2_values and z_values arrays
        np.save(os.path.join(output_folder, 'R2_values.npy'), R2_values)
        np.save(os.path.join(output_folder, 'z_values.npy'), z_values)

        print(f'Total computation time: {time.time() - start_time:.2f} seconds')
